In [1]:
!pip install langgraph langgraph-prebuilt langgraph-sdk langsmith langchain-community langchain-core langchain-openai notebook tavily-python langchain-google-genai


- Learn how to create Tools in LangGraph
- How to create a ReAct Graph
- Work with different types of Messages such as ToolMessages
- Test out robustess of our graph

In [11]:
from typing import Annotated, Sequence, TypedDict #Type annotations for type hints
import os
from dotenv import load_dotenv
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.messages import BaseMessage #The foundational class for all message types in LangGraph
from langchain_core.messages import SystemMessage # Message for providing instructions to the LLM
from langchain_core.messages import ToolMessage #Passes data back to LLM after it calls a tool such as the content and the tool name
from langchain_google_genai import GoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import BaseTool
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode


-annotated is ,, and TypedDict is used for creating dictionaries with specific key-value pairs.


- **Annotated** - Provides additional context without affecting the type itself, used to add metadata to types
```
from typing import Annotated

def greet(name: Annotated[str, "must be non-empty"]) -> None:
    print(f"Hello, {name}!")

greet("Pratham")
```

- **Sequence** - Used for type hinting sequences, To automatically handle the state updates for sequences such as by adding new message to a chat history
Sequence is an abstract type that includes any ordered collection that supports:

Indexing (`obj[0]`)

Iteration (`for item in obj`)

Length (`len(obj)`)

So things like:

- list

- tuple

- str

...all qualify as a Sequence

```
from typing import Sequence

def print_items(items: Sequence[str]):
    for item in items:
        print(item)

print_items(["apple", "banana", "mango"])  # list
print_items(("cat", "dog", "fish"))        # tuple
```

In [3]:
load_dotenv()

True

## Reducer Function
- Rule that controls how updates from nodes are combined with the existing state
- Tells us how to merge new data with the existing state
- without a reducer, updates would have replaced the existing value entirely

### without a reudcer

```
state = {"message": ["hi"]}
update = {"message": ["hello"]}
new_state = {"message": ["hello"]}  # This would replace the existing value entirely
```
### with a reducer
`
state = {"message": ["hi"]}
update = {"message": ["hello"]}
new_state = {"message": ["hi","hello"]} # This merges the new value with the existing value
`

In [4]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage],add_messages]

In [ ]:
@tool
def add(a:int, n:int):
    """Adds a number to the current total."""
    return a + n

tools = [add]

model = ChatOpenAI(model = "gpt-4o",openai_api_key = os.environ['OPEN_AI_KEY']).bind_tools(tools)

def model_call(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(
        content="You are a helpful AI assistant. You will answer questions and perform tasks as requested by the user."
    )
    response = model.invoke([system_prompt] + state["messages"])
    return {"messages": state["messages"] + [response]}



In [ ]:
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "continue"
    return "end"    

In [27]:
def tool_executor(state: AgentState) -> AgentState:
    last_message = state["messages"][-1]
    new_messages = list(state["messages"])

    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        for call in last_message.tool_calls:
            tool_name = call["name"]
            args = call["args"]
            for tool in tools:
                if tool.name == tool_name:
                    result = tool.invoke(args)
                    new_messages.append(
                        ToolMessage(tool_call_id=call["id"], content=str(result))
                    )
    return {"messages": new_messages}

In [29]:
graph = StateGraph(AgentState) 
graph.add_node("our_agent",model_call) 

tool_node = ToolNode(tools=tools)
graph.add_node("tool_node", tool_executor)

graph.add_edge(START, "our_agent") #Start the graph with our_agent node

graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue":"tool_node",
        "end": END
    }
)

graph.add_edge("tool_node", "our_agent")
graph.add_edge("tool_node", END) #End the graph after tool_node
app = graph.compile()



In [20]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            print(message.pretty_print())

In [30]:
inputs = {
    "messages": [
       ("user", "Add 40 + 12 and then multiply the result by 6.Also tell me a joke")
    ]
}
print_stream(app.stream(inputs, stream_mode="values")) #Stream the output of the app with the inputs provided

('user', 'Add 40 + 12 and then multiply the result by 6.Also tell me a joke')
================================== Ai Message ==================================
Tool Calls:
  add (call_JK2MTR8vtyaZ82QnGt3U4jl5)
 Call ID: call_JK2MTR8vtyaZ82QnGt3U4jl5
  Args:
    a: 40
    n: 12
None
================================= Tool Message =================================

52
None
================================== Ai Message ==================================

The result of adding 40 and 12 is 52. Now, I will multiply 52 by 6.

As for the joke: 

Why don't scientists trust atoms? Because they make up everything! 

Now, let's complete the multiplication task.
Tool Calls:
  add (call_xfdOCyjNIahxLLEQldmGU4xG)
 Call ID: call_xfdOCyjNIahxLLEQldmGU4xG
  Args:
    a: 52
    n: 52
None
================================= Tool Message =================================

104
None
================================== Ai Message ==================================
Tool Calls:
  add (call_FKkNcbquHkOhkBVOlkujc5GI)